In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
samplesize = 5

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.preprocessing import MinMaxScaler

#rotate a list by given offset, (negative offset, right to left) 
def rotateList(items, offset):
    return np.roll(items,offset, axis = 0)

def secondToMillisecond(val):
    return val*1000

def gaussiantimeseries(limit = 60, interval = 2):
    data = {}
    for t in range(0,limit,interval):
        data[t]=random.normal(loc = random.uniform(1,100), scale =1.5, size =1000).astype(float)
    return data

def createsignal(limit = 60, interval = 2):
    signal1, signal2, offset = [],[],[]
    limit = secondToMillisecond(limit)
    offset = random.randint(-10,10)
    data = gaussiantimeseries(limit, interval)
    for t in data:
        dist = data[t]
        signal1.append(random.choice(dist,size=samplesize,replace=True))
        signal2.append(random.choice(dist,size=samplesize,replace=True))
    # print(array(signal1).shape)
    return array(signal1), array(signal2), array(rotateList(signal2,offset)), offset

def createsignals(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    data = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        data.append([sig1, sig2original, sig2, offset])
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append((train_data, torch.FloatTensor([offset])))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append((train_data_normalized, torch.FloatTensor([offset])))
    return input_seq, data

def createsignalsfortensor(num = 50, limit = 60, interval = 2, normalize = False):
    input_seq = []
    labels = []
    for i in range(num):
        sig1, sig2original, sig2, offset = createsignal(limit = limit)
        train_data = np.hstack((sig1,sig2)).astype(float)
        #train data without normalization
        if not normalize:
            train_data = torch.FloatTensor(train_data)
            input_seq.append(train_data)
            labels.append(torch.FloatTensor([offset]))
        else:
            scaler = MinMaxScaler(feature_range=(-1, 1))
            train_data_normalized = scaler.fit_transform(train_data)
            train_data_normalized = torch.FloatTensor(train_data_normalized)
            input_seq.append(train_data_normalized)
            labels.append(torch.FloatTensor([offset]))
    return input_seq, labels
# print(createsignal())

In [ ]:
train_input, train_label = createsignalsfortensor(num=1000, limit = 60, interval = 1, normalize = False)
# val_input, val_label = createsignalsfortensor(num=1)
test_input, test_label = createsignalsfortensor()

In [ ]:
a,b,c,d=createsignal(limit=1)

In [ ]:
print(f'Signal 1 \n{a}')
print(f'Signal 2 \n{b}')
print(f'Signal 2 offset \n{c}')
print(f'Offset {d}')

Signal 1 
[[28.55552299 28.24295628 29.23562057 27.86705902 26.58140591]
 [35.17129724 34.61104389 37.38758923 35.55906134 34.94916251]
 [66.77817908 66.82845959 69.3020889  69.60727155 69.42816628]
 ...
 [33.21126144 36.84982436 35.38074228 34.81057947 33.861907  ]
 [15.6752133  13.21899282 15.67185906 13.44093631 13.2141468 ]
 [28.35288561 30.78272849 31.65168202 31.05847492 26.54680626]]
Signal 2 
[[28.85310461 27.66968648 27.75387231 26.726153   28.95228821]
 [33.06658324 35.99979233 34.48708171 36.87443041 35.80069375]
 [67.76490463 70.70975232 69.57155142 67.5991584  66.05685596]
 ...
 [35.31507666 33.21463511 35.05195121 32.99777879 35.12076871]
 [17.75065692 11.10521035 14.72847647 12.67583425 13.50143784]
 [30.94754784 32.86124224 32.05077786 31.08017181 30.06895924]]
Signal 2 offset 
[[31.08017181 30.06895924 28.85310461 27.66968648 27.75387231]
 [26.726153   28.95228821 33.06658324 35.99979233 34.48708171]
 [36.87443041 35.80069375 67.76490463 70.70975232 69.57155142]
 ...
 

In [ ]:
np.roll(b,2, axis=0)

array([[17.75065692, 11.10521035, 14.72847647, 12.67583425, 13.50143784],
       [30.94754784, 32.86124224, 32.05077786, 31.08017181, 30.06895924],
       [28.85310461, 27.66968648, 27.75387231, 26.726153  , 28.95228821],
       ...,
       [51.78469061, 51.31706766, 50.07896765, 49.46005002, 51.96151397],
       [15.3433613 , 16.76198549, 15.96871145, 17.41197953, 16.91794492],
       [35.31507666, 33.21463511, 35.05195121, 32.99777879, 35.12076871]])

In [2]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        self.labels = labels
        self.data = data

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [ ]:
#prepare data first time
train_data = CustomDataset(train_input,train_label)
# val_data = CustomDataset(test_input,test_label)
test_data = CustomDataset(test_input,test_label)

batch_size = 10

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
# val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

NameError: ignored

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
torch.save(train_data, '/content/drive/My Drive/DummyGaussianData/train_data.pt')

In [ ]:
torch.save(test_data, '/content/drive/My Drive/DummyGaussianData/test_data.pt')

In [9]:
train_data.__getitem__(0)[0].shape

torch.Size([30000, 10])

In [ ]:
temp= torch.load('/content/drive/My Drive/DummyGaussianData/train_data.pt')

In [ ]:
temp.__getitem__(0)

(tensor([[48.0051, 45.7944, 48.5005,  ..., 47.1542, 91.9092, 92.9428],
         [95.4821, 95.1396, 94.4093,  ..., 94.9814, 31.4947, 31.8121],
         [34.2531, 34.6787, 34.2960,  ..., 32.0498, 50.4934, 51.2438],
         ...,
         [47.7550, 46.7082, 48.8083,  ..., 47.1014, 99.0546, 95.7144],
         [95.9268, 96.1973, 98.4948,  ..., 96.3217, 13.5785, 13.9686],
         [18.0889, 15.1374, 15.7893,  ..., 15.6744, 47.3348, 48.3199]]),
 tensor([-2.]))

In [4]:
#load from drive
train_data = torch.load('/content/drive/My Drive/DummyGaussianData/train_data.pt')
test_data = torch.load('/content/drive/My Drive/DummyGaussianData/test_data.pt')
batch_size = 10

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [8]:
train_data.shape

AttributeError: ignored

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size=2, output_size = 1, hidden_dim = 512, n_layers = 2, drop_prob=0):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        # x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        # out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        # print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [6]:
model = LSTM(samplesize*2)
model.to(device)

lr = 0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [7]:
epochs = 25
counter = 0
print_every = 100
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    # print(batch_size)
    print("Epoch {}/{}".format(i+1, epochs))
    print('-' * 20)
    epoch_loss = 0.0
    h = model.init_hidden(batch_size)
    model.zero_grad()
    for inputs, labels in train_loader:
        # print(inputs.shape)
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)

        output, h = model(inputs, h)
        # print("a",output.shape)
        # print("b",labels.squeeze().shape)
        optimizer.zero_grad()
        loss = criterion(output, labels.squeeze())
        # print(output)
        # print(labels)
        # break
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += output.shape[0] * loss.item()
        if counter%print_every == 0:
        #     # print(batch_size)
        #     val_h = model.init_hidden(batch_size)
        #     val_losses = []
        #     model.eval()
        #     for inp, lab in val_loader:
        #         val_h = tuple([each.data for each in val_h])
        #         inp, lab = inp.to(device), lab.to(device)
        #         out, val_h = model(inp, val_h)
        #         val_loss = criterion(out, lab.squeeze())
        #         val_losses.append(val_loss.item())
                
            # model.train()
            print("Epoch: {}/{}".format(i+1, epochs),
                  "Step: {}".format(counter),
                  "Loss: {:.6f}".format(loss.item()))
            #       "Val Loss: {:.6f}".format(np.mean(val_losses)))
            # if np.mean(val_losses) <= valid_loss_min:
            #     # torch.save(model.state_dict(), './state_dict.pt')
            #     print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(valid_loss_min,np.mean(val_losses)))
            #     valid_loss_min = np.mean(val_losses)
    # break
    print("Epoch {} Loss {}".format(i+1, epoch_loss / train_data.__len__()))

Epoch 1/25
--------------------
Epoch: 1/25 Step: 100 Loss: 21.870987
Epoch 1 Loss 32.908212203979495
Epoch 2/25
--------------------
Epoch: 2/25 Step: 200 Loss: 42.053051
Epoch 2 Loss 32.676111211776735
Epoch 3/25
--------------------
Epoch: 3/25 Step: 300 Loss: 46.768131
Epoch 3 Loss 32.53796642303467
Epoch 4/25
--------------------
Epoch: 4/25 Step: 400 Loss: 33.699238
Epoch 4 Loss 32.68483510017395
Epoch 5/25
--------------------
Epoch: 5/25 Step: 500 Loss: 23.495514
Epoch 5 Loss 32.0699622631073
Epoch 6/25
--------------------
Epoch: 6/25 Step: 600 Loss: 25.656757
Epoch 6 Loss 31.014208173751832
Epoch 7/25
--------------------
Epoch: 7/25 Step: 700 Loss: 22.985779
Epoch 7 Loss 29.57295407295227
Epoch 8/25
--------------------
Epoch: 8/25 Step: 800 Loss: 29.672606
Epoch 8 Loss 28.761074028015138
Epoch 9/25
--------------------
Epoch: 9/25 Step: 900 Loss: 52.611511
Epoch 9 Loss 27.619890127182007
Epoch 10/25
--------------------
Epoch: 10/25 Step: 1000 Loss: 17.291422
Epoch 10 Loss 

KeyboardInterrupt: ignored

In [ ]:
for inputs, labels in test_loader:
  print(f'Inputs {inputs}')
  print(f'Offset {labels}')
  break

Inputs tensor([[[ 74.6624,  75.3381,  76.4773,  ...,  76.3646,  76.8026,  57.8330],
         [ 56.6395,  58.9878,  55.7450,  ...,  57.5289,  58.2122,   1.3047],
         [  3.0441,   2.5610,   5.6409,  ...,   1.6372,  -0.7992, 101.9896],
         ...,
         [ 81.2813,  82.1141,  85.0519,  ...,  83.8684,  84.1401,  29.6996],
         [ 28.3823,  28.3495,  25.1601,  ...,  29.2779,  28.4408,  22.7246],
         [ 21.4903,  22.4944,  22.0984,  ...,  20.6886,  17.1397,  75.4028]],

        [[ 44.2611,  42.2968,  45.1024,  ...,  96.6998,  97.4612,  95.5610],
         [ 97.1377,  95.5053,  94.9786,  ...,  78.7059,  73.2124,  75.5708],
         [ 74.5634,  74.1338,  73.5546,  ...,  83.3581,  82.4113,  82.4130],
         ...,
         [ 22.7079,  24.1410,  25.6830,  ...,  79.7743,  77.3991,  80.5215],
         [ 80.7323,  79.7478,  79.2406,  ...,   9.1928,  10.0205,  10.1707],
         [  7.9283,  10.4839,  10.4043,  ...,  42.5363,  44.1317,  43.0733]],

        [[ 53.0811,  56.1348,  57.182

In [ ]:
#saving model to drive

torch.save(model.state_dict(), f'/content/drive/My Drive/DummyGaussianData/model_58%.pt')

In [ ]:
#loading model from drive
model = LSTM(samplesize*2)
model.to(device)
model.load_state_dict(torch.load('/content/drive/My Drive/DummyGaussianData/model.pt'))
lr = 0.0001
criterion = nn.MSELoss()

In [10]:
test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)
counter = 0
model.eval()
for inputs, labels in test_loader:
    counter+=1
    print(f'Starting Batch {counter}')
    print('-' * 20)
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    # print(inputs)
    output, h = model(inputs, h)
    test_loss = criterion(output, labels.squeeze())
    test_losses.append(test_loss.item())
    pred = torch.round(output)  # Rounds the output to 0/1
    print(f" Offset {labels.squeeze().data},\n Prediction {output.data},\n Prediction rounded {pred}")
    correct_tensor = pred.eq(labels.squeeze().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Starting Batch 1
--------------------
 Offset tensor([-3.,  8., -7., -4., -5.,  8.,  7., -7.,  2., -3.], device='cuda:0'),
 Prediction tensor([-0.5661, -0.1455, -3.4906, -0.9554, -3.5829,  1.9805, -2.9214, -3.8930,
         2.2261, -3.9950], device='cuda:0'),
 Prediction rounded tensor([-1., -0., -3., -1., -4.,  2., -3., -4.,  2., -4.], device='cuda:0',
       grad_fn=<RoundBackward>)
Starting Batch 2
--------------------
 Offset tensor([ 1.,  2.,  1., -7., -5., -9., -8.,  2.,  2.,  4.], device='cuda:0'),
 Prediction tensor([-1.3561, -0.6281,  1.8744,  0.3958, -2.9294, -0.0912, -3.2416,  2.7599,
         2.6183,  2.4276], device='cuda:0'),
 Prediction rounded tensor([-1., -1.,  2.,  0., -3., -0., -3.,  3.,  3.,  2.], device='cuda:0',
       grad_fn=<RoundBackward>)
Starting Batch 3
--------------------
 Offset tensor([  7., -10.,   3.,   6.,   3.,   1.,   0.,   8.,   4.,  -4.],
       device='cuda:0'),
 Prediction tensor([-2.3742, -3.5964,  2.9530,  2.5391,  2.2023, -1.7423, -2.6908,  